In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score


#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [4]:
TS_Fresh_setting, connectivity_setting = 'Minimal' , 'beta'

In [13]:
data_dict = {}
for data_type in ['N1','REM', 'N2', 'N3', 'Wake']:
    data_dict[data_type] = {}
    # Load the features 
    load_path = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
    load_path_bpw = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
    load_path_max_freqs = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Gen_New_Features/generated_feats/'
    
    loaded_data = {}
    
    X_y_groups = {}
        
    X_bpw = pd.read_hdf(load_path_bpw + data_type +  'six_second_freq_df.h5', key='df', mode='r')
    y = pd.read_hdf(load_path + data_type +  '_y.h5', key='df', mode='r') 
    groups = pd.read_hdf(load_path + data_type +  '_groups.h5', key='df', mode='r')
    X_bpw, y , groups = X_bpw.reset_index(drop = True) , y.reset_index(drop = True) , groups.reset_index(drop = True)
    
    X_max_freqs = pd.read_hdf(load_path_max_freqs + data_type +  'six_second_max_freq_stats_df.h5', key='df', mode='r')
    X_max_freqs = X_max_freqs.reset_index(drop = True)
    
    X = pd.concat([X_bpw , X_max_freqs], axis = 1)
    
    #Transform the X into TS_Fresh Features___
    # 1. Convert to TS_Fresh format Dataframe 
    ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(X, ts_cols = list(X.columns))
    
    # 2. Extract TS_Fresh Features from the dataframe
    if TS_Fresh_setting == 'Minimal':
        settings = MinimalFCParameters()
    
    extracted_ts_fresh_df = extract_features(ts_fresh_df, column_id = 'id' , column_sort = 'time', default_fc_parameters=settings)
    
    # 3. Asign extract_ts_fresh_df to the variable X
    X = extracted_ts_fresh_df.copy()
    print(len(X.columns))
    #Drop columns where all values are NA 
    X = X.dropna(axis = 1)
    print(len(X.columns))
    
    #Drop columns where all values are the same
    # Find columns where all values are the same
    same_value_columns = X.columns[X.nunique() == 1]
    # Drop columns with the same values
    X = X.drop(columns=same_value_columns)
    print(len(X.columns))
    
    #___________load the appropriate connectivity features don't concatenate to non connectivity features until after dictionary generation
    
    connectivity_folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/Connectivity/'
    X_connectivity = pd.read_hdf(connectivity_folder + data_type+ '_pli__df.h5')
    
    X_connectivity = X_connectivity[[col for col in X_connectivity.columns if connectivity_setting in col]]
    
    #Generate a region to features dictionary - this will enable us to run expts regionally as before
    regional_features_dict = {}
    region_channel_dict = constants.region_to_channel_dict
    regions = list(region_channel_dict.keys())
    for region in regions:
        region_features = [col for col in X.columns if region + '_' in col]
        if len(region_features) > 0 : 
            regional_features_dict[region] = region_features + list(X_connectivity.columns)
    
    #Now concatenate the regional frequency features dataframe with the connectivity dataframe
    X = X.reset_index(drop=True)
    X_connectivity = X_connectivity.reset_index(drop=True)
    X = pd.concat([X , X_connectivity], axis = 1)
    X = X.iloc[:1,:]
    data_dict[data_type]['X'] = X
    data_dict[data_type]['regional_features_dict'] = regional_features_dict

16-Nov-23 16:59:42 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 44.67it/s]


1950
1950
1948


16-Nov-23 16:59:48 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 50.05it/s]


1950
1950
1940


16-Nov-23 16:59:56 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 41.58it/s]


1950
1950
1931


16-Nov-23 17:00:03 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 40.33it/s]


1950
1950
1927


16-Nov-23 17:00:09 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 58.64it/s]


1950
1950
1950


In [19]:
joblib.dump(data_dict, 'data_dict.pkl')

['data_dict.pkl']

In [20]:
data_dict.keys()

dict_keys(['N1', 'REM', 'N2', 'N3', 'Wake'])

In [21]:
data_dict['N1'].keys()

dict_keys(['X', 'regional_features_dict'])

In [22]:
data_dict['N1']['X']

,Delta_Central__sum_values,Delta_Central__median,Delta_Central__mean,Delta_Central__length,Delta_Central__standard_deviation,Delta_Central__variance,Delta_Central__root_mean_square,Delta_Central__maximum,Delta_Central__absolute_maximum,Delta_Central__minimum,...,beta_PO8_POz,beta_PO8_O1,beta_PO8_O2,beta_PO8_Oz,beta_POz_O1,beta_POz_O2,beta_POz_Oz,beta_O1_O2,beta_O1_Oz,beta_O2_Oz
0,19.637975,0.387197,0.357054,55.0,0.125856,0.01584,0.378586,0.662517,0.662517,0.109404,...,0.26884,0.28482,0.316346,0.264954,0.274887,0.267113,0.249838,0.292593,0.264954,0.239905


In [78]:
(15*13*10) + 1650

3600